In [1]:
using RigidBodySim
using RigidBodyDynamics

using MeshCat
using MeshCatMechanisms


import ColorTypes: RGBA
import GeometryTypes

using MAT
include("../../load_common.jl")
include("../../bullet_collision.jl")

include("manipulator_utils.jl")
include("../../stomp.jl")

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/68a213444af81a792a622d24c9ee9b29483dffff-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/68a213444af81a792a622d24c9ee9b29483dffff-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Loading HttpServer methods...


LoadError: LoadError: LoadError: [91mUndefVarError: Robot not defined[39m
while loading /home/marco/scp_se3/src/test/manipulator/../../TrajectoryOptimization/robot/pandabot.jl, in expression starting on line 8
while loading /home/marco/scp_se3/src/test/manipulator/manipulator_utils.jl, in expression starting on line 6

### Explanation on Motion

To create an animation of a robot, use the setanimation!(MechanismVisualizer, t, q) command. t represents the timesteps, and q represents the configurations. Because we created a floating joint for the Astrobee, the configuration actually rotates and translates the robot. The configuration is represented as follows: [q1, q2, q3, q4, x, y, z] where q represents the quaternion (with [1,0,0,0] being no rotation) and [x,y,z] is the translation in the respective axis

To set the robot configuration, use the set_configuration!(state, joint, config)  [note: removing the joint section will just default to rotating/translating the robot] command. At the time of writing, it is unknown (and probably unlikely) that set_configuration! can be used by the animation :\ The Astrobee has 6 joints 




In [2]:
using PandaRobot
vis = Visualizer()

pan = Panda()
mvis = MechanismVisualizer(
    pan.mechanism,
    URDFVisuals(PandaRobot.urdfpath(), package_path=[dirname(dirname(PandaRobot.urdfpath()))]),
    vis);

open(vis)
sleep(1)

q = [ 
    [0;0;0;0;0;0;0;0;0],
    20*[0.0;0.0;0.0;0.1;0;0;0;0;0],
    [0;0;0;0;0;0;0;0;0],
    [0;0.5;0.7;0.5;0.3;0.8;0.2;0.02;0.02]
    ]


setanimation!(mvis, 1:length(q), q)
pan.mechanism

Listening on 0.0.0.0:8700...


Info: Serving MeshCat visualizer at http://127.0.0.1:8700


Created new window in existing browser session.


Spanning tree:
Vertex: world (root)
  Vertex: panda_link0, Edge: panda_link0_to_world
    Vertex: panda_link1, Edge: panda_joint1
      Vertex: panda_link2, Edge: panda_joint2
        Vertex: panda_link3, Edge: panda_joint3
          Vertex: panda_link4, Edge: panda_joint4
            Vertex: panda_link5, Edge: panda_joint5
              Vertex: panda_link6, Edge: panda_joint6
                Vertex: panda_link7, Edge: panda_joint7
                  Vertex: panda_link8, Edge: panda_joint8
                    Vertex: panda_hand, Edge: panda_hand_joint
                      Vertex: panda_leftfinger, Edge: panda_finger_joint1
                      Vertex: panda_rightfinger, Edge: panda_finger_joint2
No non-tree joints.

The following code iterates through all of the links to add points based on each link's local frame. 

- this_link_frame is the frame of the local link. This is obtained by the function default_frame(RigidBody)
- this_link_point creates a point centered at the origin of the local frame. The 3 numbers afterwards translate the point in the x, y, and z direction of the local frame
- this_link_radius represents the radius size


For visualization purposes, the "function setelement!(MechanismVisualizer,Point3D, int radius, string name)" can be used. Note, renaming a point the same name as a previously used point will replace the old point

Joints can be found by the function: "findjoint(mechanism, string link_name_from_urdf)". The frames of the joint can be recovered by "frame_before(joint)" or "frame_after(joint)"

In [3]:
#Interestingly, RigidBodyDynamics features both kinematics and dynamics library; you can call mass matrix from it
#Implies Bullet should have similar functionality; an issue might be translating the information from one to the other
#Look into this

#setelement!(mvis, Point3D(frame_after(findjoint(pan.mechanism,"panda_link0_to_world")),0.0,0.0,0.0), radius, panda_link0_name)

#http://www.juliarobotics.org/RigidBodyDynamics.jl/stable/spatial.html#RigidBodyDynamics.Spatial.Point3D

link_frame = CartesianFrame3D[]
point = Point3D[]
radius = Float64[]
i = 1;
for link in RigidBodyDynamics.bodies(pan.mechanism)
    this_link_frame = RigidBodyDynamics.default_frame(link)
    this_link_point = RigidBodyDynamics.Point3D(this_link_frame,0.0,0.0,0.1)
    this_link_radius = 0.1

    
    push!(link_frame, this_link_frame)
    push!(point,this_link_point)
#     push!(radius,this_link_radius)
    
#   for visualization
#     MeshCatMechanisms.setelement!(mvis, this_link_point, string("point_",i))
#     MeshCatMechanisms.setelement!(mvis, this_link_point, this_link_radius, string("point_",i))
    i=i+1

end

RigidBodyDynamics.bodies(pan.mechanism)
pan.mechanism

Spanning tree:
Vertex: world (root)
  Vertex: panda_link0, Edge: panda_link0_to_world
    Vertex: panda_link1, Edge: panda_joint1
      Vertex: panda_link2, Edge: panda_joint2
        Vertex: panda_link3, Edge: panda_joint3
          Vertex: panda_link4, Edge: panda_joint4
            Vertex: panda_link5, Edge: panda_joint5
              Vertex: panda_link6, Edge: panda_joint6
                Vertex: panda_link7, Edge: panda_joint7
                  Vertex: panda_link8, Edge: panda_joint8
                    Vertex: panda_hand, Edge: panda_hand_joint
                      Vertex: panda_leftfinger, Edge: panda_finger_joint1
                      Vertex: panda_rightfinger, Edge: panda_finger_joint2
No non-tree joints.

In [4]:
# Jacobian_IK_and_Control notebook in RigidBodyDynamics is a good example 

#bodies(mechanism) returns the links in the chain
body = RigidBodyDynamics.bodies(pan.mechanism)[6]
#root_frame is the world frame of the entire mechanism
world = RigidBodyDynamics.root_frame(pan.mechanism)

#A graph of your joints that you care about, from one joint to another joint. E.g. Joint 1 -> Joint 6
p = RigidBodyDynamics.path(pan.mechanism, root_body(pan.mechanism),body)

#How to create a Mechanism State and enact it upon the robot
state = RigidBodyDynamics.MechanismState(pan.mechanism)
set_configuration!(state,[0;0.4;0.8;0.3;0.5;1.0;0.4;0.02;0.02])
set_configuration!(mvis, configuration(state))

#Create a point in the world frame
desired_point = Point3D(world,0.5,0.0,2.0)

#Get the jacobian for the point given the joints we care about, the state it's in, and the point
Jp = point_jacobian(state,p,RigidBodyDynamics.transform(state,desired_point,world))
println(Jp)
# Jp = point_jacobian(state,p,RigidBodyDynamics.transform(state,point[6],point[6].frame))
# println(RigidBodyDynamics.bodies(pan.mechanism))

# End effector 
EE_id = 11
J_EE = point_jacobian(state,p,RigidBodyDynamics.transform(state,point[EE_id],world))




EE_link = RigidBodyDynamics.bodies(pan.mechanism)[EE_id]

    EE_link_frame = RigidBodyDynamics.default_frame(EE_link)
    EE_link_point = RigidBodyDynamics.Point3D(EE_link_frame,0.0,0.0,0.1)
    EE_link_radius = 0.01

    
#     push!(link_frame, this_link_frame)
#     push!(point,EE_link_point)
#     push!(radius,this_link_radius)
    
#   for visualization
#     MeshCatMechanisms.setelement!(mvis, this_link_point, string("point_",i))
MeshCatMechanisms.setelement!(mvis, EE_link_point, EE_link_radius, string("end_effector"))
#     setobject!(mvis, 
#         Object(GeometryTypes.HyperSphere(GeometryTypes.Point3f0(this_link_point), 0.05f0)
#             ,MeshBasicMaterial(color=RGBA(0,1.0,0.,0.6))))
# RigidBodyDynamics.transform(this_link_point,desired_point,world)
    EE_link_point
        point_in_world = RigidBodyDynamics.transform(state, EE_link_point, world)
# [2 3 4;5 6 7;4 1 5]*point_in_world.v

    
#Not sure how transform(configuration, point, frame) is transforming the point. Is it to the end-effector's frame, or the local frame?
#point_jacobian -> if we give an entire manipulator configuration, will this return based on our shorter chain path (probably does)

# RigidBodyDynamics.bodies(pan.mechanism)["panda_link1"]
RigidBodyDynamics.findbody(pan.mechanism, "panda_link1")


RigidBodyDynamics.Spatial.PointJacobian{Array{Float64,2}}([0.0 1.667 -7.80829e-13 -0.990757 0.0 0.0 0.0 0.0 0.0; 0.5 2.44829e-12 -0.18863 -1.01443 0.0 0.0 0.0 0.0 0.0; 0.0 -0.5 4.0929e-13 0.186632 0.0 0.0 0.0 0.0 0.0], CartesianFrame3D: "world" (id = 0))


RigidBody: "panda_link1"

In [5]:
# Inverse Kinematics example
q0 = [0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0]
q = [ 
    q0,
#     [0;0.5;0.7;0.5;0.3;0.8;0.2;0.02;0.02]
    ]
state = RigidBodyDynamics.MechanismState(pan.mechanism)
set_configuration!(state,q[1])
set_configuration!(mvis, configuration(state))



# End effector 
EE_id = 11
# J_EE = point_jacobian(state,p,RigidBodyDynamics.transform(state,point[EE_id],world))

EE_link = RigidBodyDynamics.bodies(pan.mechanism)[EE_id]
    EE_link_frame = RigidBodyDynamics.default_frame(EE_link)
    EE_link_point = RigidBodyDynamics.Point3D(EE_link_frame,0.0,0.0,0.1)
    EE_link_radius = 0.01
#   for visualization
#     MeshCatMechanisms.setelement!(mvis, this_link_point, string("point_",i))

MeshCatMechanisms.setelement!(mvis, EE_link_point, EE_link_radius, string("end_effector"))

EE_point_inWorldFrame = RigidBodyDynamics.transform(state, EE_link_point, world)
println(EE_point_inWorldFrame)

hand_body = findbody(pan.mechanism, "panda_hand")



# Choose a desired location. We'll move the tip of the arm to
# [0.5, 0, 2]
desired_tip_location = Point3D(root_frame(pan.mechanism), 0.2, 0.0, 1.2)
# Run the IK, updating `state` in place
println(configuration(state) )
println("EE_point_inWorldFrame: $EE_point_inWorldFrame")
jacobian_transpose_ik!(state, hand_body, EE_point_inWorldFrame, desired_tip_location)
println(configuration(state))
push!(q, state.q)


println(" ")

# desired_tip_location = Point3D(root_frame(pan.mechanism), 0.5, 0.1, 0.4)
# # Run the IK, updating `state` in place
# jacobian_transpose_ik!(state, hand_body, EE_point_inWorldFrame, desired_tip_location)
# push!(q, state.q)
# q = [ 
#     q,
#     state.q
#     ]
setanimation!(mvis, 1:length(q), q)
# q








Point3D in "world": [0.088, -1.69422e-12, 0.826]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
EE_point_inWorldFrame: Point3D in "world": [0.088, -1.69422e-12, 0.826]


LoadError: [91mUndefVarError: jacobian_transpose_ik! not defined[39m

In [6]:
# ********************************************
# *****  INITIALIZE ROBOT / MECHANISM   ******
# ********************************************
# Initial configuration/state
q0 = [0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0;0.0]
state = RigidBodyDynamics.MechanismState(pan.mechanism)
set_configuration!(state,q0)
set_configuration!(mvis, configuration(state))
# Final desired position of EE
EE_desired_inWorldFrame = Point3D(root_frame(pan.mechanism), 0.7, 0.0, 0.15)


# *****************
# Environment frame
world = RigidBodyDynamics.root_frame(pan.mechanism)



# ************
# End effector 
EE_id = 11
# J_EE = point_jacobian(state,p,RigidBodyDynamics.transform(state,point[EE_id],world))
# EE_link = RigidBodyDynamics.bodies(pan.mechanism)[EE_id]
EE_link = findbody(pan.mechanism, "panda_hand")
    EE_link_frame = RigidBodyDynamics.default_frame(EE_link)
    EE_link_point = RigidBodyDynamics.Point3D(EE_link_frame,0.0,0.0,0.1)
#      set_configuration!(state,q0+0.1)
    EE_point_inWorldFrame = RigidBodyDynamics.transform(state, EE_link_point, world)
    EE_link_radius = 0.02
    EE_path = RigidBodyDynamics.path(pan.mechanism, root_body(pan.mechanism),EE_link)
#     EE_Jac = point_jacobian(state,EE_path,RigidBodyDynamics.transform(state,point[EE_id],world))
#     println(EE_point_inWorldFrame)
# Vizualize
MeshCatMechanisms.setelement!(mvis, EE_link_point, EE_link_radius, string("end_effector"))

println(EE_link_point)
println(EE_point_inWorldFrame)
println(RigidBodyDynamics.transform(state, EE_link_point, world))
println(RigidBodyDynamics.transform(state, EE_point_inWorldFrame, world))

Point3D in "after_panda_hand_joint": [0.0, 0.0, 0.1]
Point3D in "world": [0.088, -1.69422e-12, 0.826]
Point3D in "world": [0.088, -1.69422e-12, 0.826]
Point3D in "world": [0.088, -1.69422e-12, 0.826]


In [7]:
# ********************************************
# ******     INITIALIZE ENVIRONMENT     ******
# ********************************************

import StaticArrays: SVector
using GeometryTypes
body_radius = 0.3f0

# Define end-effector as robot, it's a sphere
# btrobot_ = BT.geometry_type_to_BT(pan) # bullet representation of astrobee
EE_sphere = GeometryTypes.HyperSphere(Point3f0(0.,0.,0.), body_radius) # radius = 0.3m

bt_EE_ = BT.geometry_type_to_BT(EE_sphere) # bullet representation of astrobee
btworld_ = BT.collision_world(SVector{3}(-100.,-100.,-100.), 
        SVector{3}(100.,100.,100.)) # workspace bounds

env = BT.BulletStaticEnvironment(bt_EE_, btworld_) # bullet representation of environment
fieldnames(env) # env.convex_env_components is currently empty since no obstacles have been added

# define obstacles using GeometryTypes
obstacles = []
push!(obstacles, HyperSphere(Point3f0(1.,1.,1.), 0.1f0))
push!(obstacles, HyperRectangle(Vec3f0(0.4,-0.5,0.5),Vec3f0(0.4,1.0,0.2)))

# add obstacles to Bullet environment
for obs in obstacles
    BT.add_collision_object!(env, BT.geometry_type_to_BT(obs))
end
env.convex_env_components # added four obstacles to Bullet environment


# plot scene
# vis = Visualizer()
vis[:obstacles]
for (idx,obs) in enumerate(obstacles)
    setobject!(vis[:obstacles][Symbol(string("obs",idx))], 
        Object(obs,MeshBasicMaterial(color=RGBA(1.0,0.,0.,1.))))
end
in_cell = true
in_cell ? IJuliaCell(vis) : open(vis)
println("Added obstacles")


Added obstacles


In [8]:
# ********************************************
# ******  PRE-SCRIPT - STOMP CONSTANTS  ******
# ********************************************
# include("../../stomp.jl")
using Distributions
srand(123)

# parameters
# ab = Astrobee()
weight_torques = 1e-12
weight_ref = 1e-7 # cost on distance to Q0
weight_ref = 0 # cost on distance to Q0
# body_radius = P.robot.r # 0.05
# body_mass = get(P.robot.mass, :threeD, 0) # 7.0
body_mass = 1.0
if body_mass == 0
    println("[stomp_freeflyer]: Robot mass not found and set to $body_mass")
end
body_inertia     = eye(3)
body_inertia_inv = eye(3)
epsilon_obstacles = 1e-2 # redefined later depending on discretization of distances table
delta_cost_stop = 0.05
nb_iter_max = 300
h = 10 # hyperparameter, replaces lambda (see eq. 11)
# p = 3  # statespace dimension
p = length(state.q)

N = 100 # number of discretized points (including initial/final)
NB_MOTORS = p
T = 5 # horizon, in [s] (NOTE: 100samples/sec looks like what they used in paper)
dt = T/N # timestep
K = 5 # number noise trajectories
# x0,xf = zeros(3), 2*ones(3) # initial/final positions
# Q_ROT_COEFFS = 4:6
x0,xf = EE_point_inWorldFrame.v, EE_desired_inWorldFrame.v
jacobian_transpose_ik!(state, EE_link, EE_link_point, EE_desired_inWorldFrame)
q0 = copy(q0)
qf = copy(state.q)  # initial/final configurations

# Initial trajectory (linear interpolation from initial to final conf., including those)
Q0 = zeros(p,N) # parameters
for dim in 1:p
    Q0[dim,:] = collect(linspace(q0[dim],qf[dim],N))
end

# *****************************************
Aacc = get_finite_differencing_acceleration_matrix(N+2*2, dt)
Racc = Aacc'*Aacc

# A = zeros(N,N)
# A = A - 30.0/12.0*diagm(ones(N))
# A = A + 16.0/12.0*diagm(ones(N-1),1) + 16.0/12.0*diagm(ones(N-1),-1)
# A = A - 1/12.0*diagm(ones(N-2),2) - 1/12.0*diagm(ones(N-2),-2)
# A = A/(dt^2)
A = get_finite_differencing_acceleration_matrix(N, dt)
R = A'*A
# Rinv = inv(R)
Rinv = inv(Racc)[3:end-2,3:end-2]
Rinv = (Rinv+ Rinv'); # Make it symmetrical, it's not before (rounding errors)
M = copy(Rinv)
for col_i = 1:size(M,2)
    max_col_i = maximum(M[:, col_i])
    M[:,col_i] = M[:,col_i] / (max_col_i*N)
end


LoadError: [91mUndefVarError: jacobian_transpose_ik! not defined[39m

In [9]:
# Show trajectory from initial to final point

include("manipulator_utils.jl")


println(q0)
println(qf)

set_configuration!(state,q0)
set_configuration!(mvis, configuration(state))


sleep(0.5)


set_configuration!(state,qf)
set_configuration!(mvis, configuration(state))

LoadError: LoadError: LoadError: [91mUndefVarError: Robot not defined[39m
while loading /home/marco/scp_se3/src/test/manipulator/../../TrajectoryOptimization/robot/pandabot.jl, in expression starting on line 8
while loading /home/marco/scp_se3/src/test/manipulator/manipulator_utils.jl, in expression starting on line 6

In [10]:
# Debug trajectories

include("manipulator_utils.jl")


# println(q0)
# println(qf)

# set_configuration!(state,q0)
# set_configuration!(mvis, configuration(state))


# sleep(0.5)


# set_configuration!(state,qf)
# set_configuration!(mvis, configuration(state))

# println(q0)
# println(qf)

# sleep(0.5)
println(desired_tip_location)


#     EE_point_inWorldFrame = RigidBodyDynamics.transform(state, EE_link_point, world)

jacobian_transpose_ik!(state, EE_link, EE_point_inWorldFrame,         desired_tip_location)
set_configuration!(state,qf)
set_configuration!(mvis, configuration(state))
println(state.q)

sleep(0.5)

jacobian_transpose_ik!(state, EE_link, EE_link_point,         desired_tip_location)
# set_configuration!(state,qf)
set_configuration!(mvis, configuration(state))
println(state.q)


# state = RigidBodyDynamics.MechanismState(pan.mechanism)
println(EE_point_inWorldFrame)
println(EE_link_point)
println(RigidBodyDynamics.transform(state, EE_link_point, world))
# setanimation!(mvis, 1:length(q), q)


#     setobject!(vis[:obstacles][Symbol(string("ohyeah",1))], 
#         Object(HyperSphere(Point3f0(desired_tip_location.v), 0.1f0),MeshBasicMaterial(color=RGBA(1.0,0.,0.,1.))))

LoadError: LoadError: LoadError: [91mUndefVarError: Robot not defined[39m
while loading /home/marco/scp_se3/src/test/manipulator/../../TrajectoryOptimization/robot/pandabot.jl, in expression starting on line 8
while loading /home/marco/scp_se3/src/test/manipulator/manipulator_utils.jl, in expression starting on line 6

In [11]:
# include("../../stomp.jl")
# compute distances to all objects and save results in a table
if(!isdefined(:DIST2OBS_MAT))
    # Get manipulator limits
    q0 = [0.0;0.0;0.0;0.0;0.0;pi;0.0;0.0;0.0]
    set_configuration!(state,q0)
    set_configuration!(mvis, configuration(state))
    lims_up = RigidBodyDynamics.transform(state, EE_link_point, world)


    X_MIN, X_MAX = (-maximum(lims_up.v)-(2*maximum(lims_up.v))/6)*ones(3), 
                   ( maximum(lims_up.v)+(2*maximum(lims_up.v))/6)*ones(3)

    NX = 131 # number of points along each dimension
    DX = (X_MAX[1]-X_MIN[1])
    dx = DX/(NX-1)
    println("The distances table will have a precision of $(dx/2) meters")
    grid_precision = dx/2
    
    # table with all signed euclidean distances to the closest obstacle for all points
    DIST2OBS_MAT = zeros(NX,NX,NX)
    DIST2OBS_MAT = create_table_distances(NX, X_MIN, DX, env);
end

The distances table will have a precision of 0.012717948717948718 meters


LoadError: [91mUndefVarError: create_table_distances not defined[39m

In [12]:
# get_dist2obs([1.,1.,1.], DIST2OBS_MAT, NX, X_MIN, DX)
env

BT.BulletStaticEnvironment((class btCollisionObject *) @0x000000000ab850a0
, Cxx.CppPtr{Cxx.CxxQualType{Cxx.CppBaseType{:btCollisionObject},(false, false, false)},(false, false, false)}[(class btCollisionObject *) @0x0000000009a25770
], (class btCollisionWorld *) @0x000000000c28f720
, Cxx.CppPtr{Cxx.CxxQualType{Cxx.CppBaseType{:btCollisionObject},(false, false, false)},(false, false, false)}[(class btCollisionObject *) @0x000000000c4f3620
, (class btCollisionObject *) @0x000000000c3de7b0
], 0)

In [13]:
# include("../../stomp.jl")
# ********************************************
# ****** MAIN SCRIPT - IMPLEMENTS STOMP ******
# ********************************************
# note: for all variables, subscripts are in this order: dimension(p) x samples(N) x trajectoryId(K)
Dist = MvNormal(zeros(N),Rinv) # create normal distribution
Q = copy(Q0) # configuration
Qperturbed = zeros(p,N,K)
Qbest = zeros(p,N,K)
Qall  = zeros(p,N,2*K)
id_Qsave = 1
for kidx = 1:K
    Qbest[:,:,kidx] = Q0
end
velocities_mag = compute_velocities_mag(Q)
torques = compute_torques(Q, NB_MOTORS, body_mass, body_inertia)

Jtot = 0+delta_cost_stop+1e-5; 
Jtot = get_total_jointsAccels_cost(Q)
for nidx = 1:N
    Jtot = Jtot + cost(Q[:,nidx],Q0,nidx, nidx, epsilon_obstacles+grid_precision, body_radius,
                       velocities_mag, torques, weight_torques, weight_ref, 
                       DIST2OBS_MAT, NX, X_MIN, DX)
end
println("Initial cost: $(Jtot/N)")
Jprev = 0
iter_i = 1;
Qk = zeros(p,N,nb_iter_max)
Jk = get_total_obstacles_cost(Q, epsilon_obstacles+grid_precision, body_radius, DIST2OBS_MAT, NX, X_MIN, DX) + 
                get_total_torques_cost(Q, NB_MOTORS, weight_torques, body_mass, body_inertia) + 
                get_total_refDeviation_cost(Q, Q0, weight_ref)
Jcost = []
Qk[:,:,1] = copy(Q0)

println("Starting Stomp main loop")
tic()
# while (abs(Jprev-Jtot) >= delta_cost_stop) && (iter_i <= nb_iter_max)
last_println = 0
while iter_i <= nb_iter_max
    if last_println>=nb_iter_max/10
#         println("Starting iter# $iter_i / $nb_iter_max")
        last_println = 0
    else
        last_println = last_println + 1
    end
        
        
    Qk[:,:,iter_i] = copy(Q)
    Jk = vcat(Jk, Jcost)
    
    Jprev = copy(Jtot)
    iter_i = iter_i+1
    
    # 1) Create K noisy trajectories
    Qperturbed = zeros(p,N,K) # configuration trajectory (in time, length N) for each dimension
    eps = zeros(p,N,K)
    for kidx in 1:K # for each trajectory
        eps[:,:,kidx] = rand(Dist,p)'
        for dim in 1:p 
#             Qperturbed[dim,:,kidx]= Q[dim,:] + ((nb_iter_max-iter_i)/nb_iter_max) * eps[dim,:,kidx]
            Qperturbed[dim,:,kidx]= Q[dim,:] + eps[dim,:,kidx]
        end
    end

    # 2) Compute costs and probability metrics
    # a)
    S = zeros(N,2*K) # includes the best ones
    Prob = zeros(N,2*K)
    Qall = cat(3,Qperturbed,Qbest)
    for kidx in 1:2*K # for each trajectory
#         velocities_mag = compute_velocities_mag(Qperturbed[:,:,kidx])
#         torques = compute_torques(Qperturbed[:,:,kidx], NB_MOTORS, body_mass, body_inertia)
        velocities_mag = compute_velocities_mag(Qall[:,:,kidx])
        torques = compute_torques(Qall[:,:,kidx], NB_MOTORS, body_mass, body_inertia)
        for nidx in 1:N # get cost for each sample
#             S[nidx,kidx] = cost(Qperturbed[:,nidx,kidx], nidx, nidx)
            S[nidx,kidx] = cost(Qall[:,nidx,kidx],Q0,nidx, nidx, epsilon_obstacles+grid_precision, body_radius,
                                velocities_mag, torques, weight_torques, weight_ref, 
                                DIST2OBS_MAT, NX, X_MIN, DX)
        end
        
    end
    
    # b)
    for nidx in 1:N
        # see eq (11)
        max_S_trajs = maximum(S[nidx,:])
        min_S_trajs = minimum(S[nidx,:])
        delta_minmax_S = max_S_trajs-min_S_trajs
        
        sum_exp_S = 0
        for kidx in 1:2*K 
            sum_exp_S = sum_exp_S + exp(-h*(S[nidx,kidx]-min_S_trajs)/delta_minmax_S)
        end
        for kidx in 1:2*K 
            Prob[nidx,kidx] = exp(-h*(S[nidx,kidx]-min_S_trajs)/delta_minmax_S) / sum_exp_S
        end
    end

    # 3-4-5) perturb parameter vector
    dQ_tilde = zeros(p,N)
    for nidx in 1:N
        for kidx in 1:K 
            dQ_tilde[:,nidx] = dQ_tilde[:,nidx] + Prob[nidx,kidx] * eps[:,nidx,kidx] # 3)
        end
        for kidx in 1:K
            dQ_tilde[:,nidx] = dQ_tilde[:,nidx] + Prob[nidx,kidx] * (Qbest[:,nidx,kidx]-Q[:,nidx]) # 3)
        end
    end
    dQ = (M * dQ_tilde')' # 4)
    Q = Q + dQ # 5)
    velocities_mag = compute_velocities_mag(Q)
    torques = compute_torques(Q, NB_MOTORS, body_mass, body_inertia)

    # 6) compute total cost
    if last_println==0
        Jacc = get_total_jointsAccels_cost(Q)
        Jcost = get_total_obstacles_cost(Q, epsilon_obstacles+grid_precision, body_radius, DIST2OBS_MAT, NX, X_MIN, DX) + 
                get_total_torques_cost(Q, NB_MOTORS, weight_torques, body_mass, body_inertia) + 
                get_total_refDeviation_cost(Q, Q0, weight_ref)
        Jtot = Jacc + Jcost
        println("Current cost at step $iter_i : $(Jtot/N)")
        println("Jacc: $Jacc, Jcost: $Jcost")
#         println("$(dQ_tilde[1,20:40])")
    end
    
    # get best samples
    costs_all = zeros(2*K)
    for traj_i = 1:2*K
        costs_all[traj_i] = get_total_obstacles_cost(Qall[:,:,traj_i], epsilon_obstacles+grid_precision, body_radius, DIST2OBS_MAT, NX, X_MIN, DX) + 
                            get_total_torques_cost(Qall[:,:,traj_i], NB_MOTORS, weight_torques, body_mass, body_inertia) + 
                            get_total_refDeviation_cost(Qall[:,:,traj_i], Q0, weight_ref)
                             #+get_total_jointsAccels_cost(Qall[:,:,traj_i]) + 
    end
    Qbest = Qall[:,:,sortperm(costs_all)[1:K]]
    
#     if (is_trajectory_collision_less(Qbest[:,:,1], epsilon_obstacles+grid_precision, body_radius, DIST2OBS_MAT, NX, X_MIN, DX))
    if (is_trajectory_collision_less(Q, epsilon_obstacles+grid_precision, body_radius, DIST2OBS_MAT, NX, X_MIN, DX))
        println("Trajectory computed at iter. $iter_i is collision-less, exiting")
        break
    end
end
elapsed_opt_time = toc()
println(elapsed_opt_time)
println("Final cost: $(Jtot/N)")

# ********************************************
# ****** END MAIN SCRIPT  -    STOMP    ******
# ********************************************

LoadError: [91mUndefVarError: Rinv not defined[39m

In [14]:
# Animate
# println(Q)



#     EE_point_inWorldFrame = RigidBodyDynamics.transform(state, EE_link_point, world)
for nidx in 1:N
    q = Q[:,nidx]
#     println(size(q))
    set_configuration!(state,q)
    set_configuration!(mvis, configuration(state))
    sleep(0.005)
end

LoadError: [91mUndefVarError: Q not defined[39m

In [15]:
# plot results
using PyPlot

# Q = Qbest[:,:,1]

PyPlot.figure()
labels=["x","y","z"]
for dim in 1:min(3,p)
    PyPlot.plot(Q[dim,:],label=labels[dim])
end
PyPlot.legend(loc="best")
PyPlot.xlabel("sample [-]"); PyPlot.title("solution")

LoadError: [91mUndefVarError: Q not defined[39m

Error handling websocket connection:
WebSockets.WebSocketClosedError("ws|server respond to OPCODE_CLOSE 1001:Going Away")